# Identifica Stringa

Il seguente codice permette di individuare gli indici e le righe in cui è presente una keyword.

I commenti all'interno dei programmi vengono esclusi da tutta la procedure

E' possibile utilizzare più parole da cercare nelle righe

E' possibile utilizzare parole per escludere le righe che lo contengono.


In [1]:
import os
import pandas as pd

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block

def verifica_presenza_parole(file_path, parole_da_verificare, parole_da_escludere, linee_con_parole):
    indici_presenti = []
    in_comment_block = False

    with open(file_path, 'r') as file:
        for numero, line in enumerate(file, start=1):
            line_originale = line.strip()

            # Elimina le parole tra /* e */
            line_senza_commenti, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)

            # CONDIZIONE
            if all(parola.lower() in line_senza_commenti.lower() for parola in parole_da_verificare) and \
               all(parola.lower() not in line_senza_commenti.lower() for parola in parole_da_escludere):
                indici_presenti.append(numero)
                # linee_con_parole.append(line_originale.lower())
                
                file_nome = os.path.basename(file.name)
                if line_originale.lower() in linee_con_parole:
                    linee_con_parole[line_originale.lower()].append(file_nome)
                else:
                    linee_con_parole[line_originale.lower()] = [file_nome]
    return indici_presenti

def crea_dataframe_presenza_parole(cartella, estensione, parole_da_verificare, parole_da_escludere):
    stringa_verificare = ','.join(parole_da_verificare)
    stringa_escludere = ','.join(parole_da_escludere)
    if len(parole_da_escludere) == 0:
        nome_colonna=stringa_verificare
    else:
        nome_colonna=stringa_verificare + '-' + stringa_escludere
    
    dati_temporanei = {'Nome File': [], nome_colonna: []}
    linee_con_parole = {}

    for root, dirs, files in os.walk(cartella):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                indici_presenti = verifica_presenza_parole(file_path, parole_da_verificare, parole_da_escludere, linee_con_parole)
                dati_temporanei['Nome File'].append(file_name)
                dati_temporanei[nome_colonna].append(indici_presenti)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('Nome File', inplace=True)
    
    df_linee = pd.DataFrame(list(linee_con_parole.items()), columns=[nome_colonna, 'Nome File'])
    # df_linee['Nome File'] = df_linee['Nome File'].apply(lambda x: list(set(x)))
    df_linee['Frequenza Programmi'] = df_linee['Nome File'].apply(len)
    df_linee.sort_values(by='Frequenza Programmi',ascending=False, inplace=True)

    return dataframe, df_linee

### Creazione Report

In [2]:
cartella_da_esplorare = r"C:\Miguel\progetti\05_Modifica Testi\01_data\PRD_9-3\ITPSAS02_DLST_CMD005"
estensione_file = ".sas"

# Chiama la funzione per creare il DataFrame
connect_to_pgm, connect_to_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['connect to'], [])

proc_import_pgm, proc_import_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['proc import'],[])

proc_export_pgm, proc_export_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['proc export'],[])

mail_pgm, mail_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%email'],[])

let_pgm, let_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%let'],[])

call_symput_pgm, call_symput_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['call symput'],[])

include_pgm, include_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%include'],[])

libname_pgm, libname_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['libname'],["%include"])

report = pd.concat([connect_to_pgm,  proc_import_pgm, proc_export_pgm, mail_pgm, let_pgm, call_symput_pgm, include_pgm, libname_pgm], axis=1)
# report.to_csv('report_Catena_20-11-2023.csv')

STATS REPORT

In [137]:
# libname_linee.to_csv('libname_linee.csv', sep=';', index=False)

In [3]:
report_length = report.map(lambda x: len(x) if isinstance(x, list) else x)
# report_length.to_csv('report_length.csv')
totali_per_colonna = report.map(lambda x: len(x) if isinstance(x, list) else 0).sum()
conteggi_per_colonna = (report.map(lambda x: len(x) if isinstance(x, list) else 0) != 0).sum()

report_means = pd.DataFrame({
    'Keyword': totali_per_colonna.index,
    'Numero Programmi' : report.shape[0],
    'Frequenza Totale': totali_per_colonna.values,
    'Frequenza Programmi' : conteggi_per_colonna.values,
    'Percentuale Programmi' : conteggi_per_colonna.values/report.shape[0]
})

# report_means.to_csv('report_means.csv', index=False, sep=';', decimal=',')

Frequenza %let

In [4]:
df_expanded = let_linee.explode(['Nome File'])
# df_expanded.to_csv('%let.csv', sep=";")

In [5]:
# Frequenza dei nomi delle macro-variabili
name_let = let_linee['%let'].apply(lambda x: x.split('=')[0])
name_let = name_let.apply(lambda x: x.replace('%let ', ''))
name_let = name_let.str.strip().str.replace('\\s+', ' ')
name_let.value_counts()

%let
jobname                28
direzione               4
nomefile                3
script1                 3
ambiente                2
percorso_out            2
infile                  2
save_versione           2
p_output                2
dir_output_xlsx         1
todaysdate              1
todaysdate1             1
ambiente_odss           1
inizio                  1
ambiente_dobit          1
data_ini                1
gg_retain               1
lista                   1
flow_status_code_03     1
percorso_i              1
percorso_o              1
flow_status_code        1
p_output_log            1
script2                 1
n_invoice_limit         1
segment                 1
ambiente_glob           1
percorso                1
nomefiledel             1
Name: count, dtype: int64

Frequenza CALL SYMPUT

In [6]:
df_expanded = call_symput_linee.explode(['Nome File'])
# df_expanded.to_csv('call_symput.csv', sep=";")

Frequenza Include

In [7]:
df_expanded = include_linee.explode(['Nome File'])
# df_expanded.to_csv('include.csv', sep=";")

Frequenza libname

In [8]:
df_expanded = libname_linee.explode(['Nome File'])

In [20]:
# df_expanded.to_csv("libname.csv",sep=';')

# Identifica Stringhe con le regex

In [8]:
import os
import pandas as pd
import re

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # Continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block

def verifica_presenza_parole(file_path, regex_pattern, parole_da_escludere, linee_con_parole):
    indici_presenti = []
    in_comment_block = False

    with open(file_path, 'r') as file:
        for numero, line in enumerate(file, start=1):
            line_originale = line.strip()

            # Elimina le parole tra /* e */
            line_senza_commenti, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)
            # regex_pattern = r'\bdata\b\s+\S+\.\S+\s*;'
            
            # Verifica la presenza di tutte le parole nella lista e l'assenza delle parole da escludere
            if re.search(regex_pattern, line_senza_commenti.lower(), re.IGNORECASE) and \
               all(parola.lower() not in line_senza_commenti.lower() for parola in parole_da_escludere):
                indici_presenti.append(numero)
                # linee_con_parole.append(line_originale.lower())
                
                file_nome = os.path.basename(file.name)
                if line_originale.lower() in linee_con_parole:
                    linee_con_parole[line_originale.lower()].append(file_nome)
                else:
                    linee_con_parole[line_originale.lower()] = [file_nome]
    return indici_presenti

def crea_dataframe_presenza_parole(cartella, estensione, regex_pattern, parole_da_escludere):
 
    nome_colonna=regex_pattern
    
    dati_temporanei = {'File': [], nome_colonna: []}
    linee_con_parole = {}

    for root, dirs, files in os.walk(cartella):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                indici_presenti = verifica_presenza_parole(file_path, regex_pattern, parole_da_escludere, linee_con_parole)
                dati_temporanei['File'].append(file_name)
                dati_temporanei[nome_colonna].append(indici_presenti)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('File', inplace=True)

    df_linee = pd.DataFrame(list(linee_con_parole.items()), columns=[nome_colonna, 'programmi'])
    df_linee['programmi'] = df_linee['programmi'].apply(lambda x: list(set(x)))
    df_linee['Frequenza pgm'] = df_linee['programmi'].apply(len)
    df_linee.sort_values(by='Frequenza pgm',ascending=False, inplace=True)

    return dataframe, df_linee

### Dove scrive...

#### DATA

In [9]:
df1_pgm, df1_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, r'\bdata\b\s+(\S+\.\S+(?:\s+\S+)*)\s*;',[])

In [10]:
# df1_pgm, df1_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, r'\bdata\b\s+\S+\.\S+\s*;',[])

In [11]:
# df1_linee.to_csv('to_elim.csv', sep=';')

### Dove legge

#### SET

In [12]:
df2_pgm, df2_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file,r'\bset\b\s+(\S+\.\S+(?:\s+\S+)*)\s*;',[])

In [13]:
df2_linee

,\bset\b\s+(\S+\.\S+(?:\s+\S+)*)\s*;,programmi,Frequenza pgm
0,set backlog2.backlog_&todaysdate1 backlog_aggi...,[PRD_1315_DRI_Lst_Daily_Backlog_Studi_Risk_v2....,1
1,set bacvlv.backlog_&todaysdate1 backlog_aggior...,[PRD_1315_DRI_Lst_Daily_Backlog_Studi_Risk_v2....,1
2,set dri_&ambiente..contact_activity_types(keep...,[PRD_1361_DRI_Ins_DWH_Activity_Complaints_v16....,1
3,set dob_&ambiente..azit_f_activities_hist;,[PRD_1361_DRI_Ins_DWH_Activity_Complaints_v16....,1
4,set dri_&ambiente..sub_codes(keep=main_code su...,[PRD_1361_DRI_Ins_DWH_Activity_Complaints_v16....,1
5,set dob_&ambiente_odss..azit_f_complaints_hist;,[PRD_1361_DRI_Ins_DWH_Activity_Complaints_v16....,1
6,set dwh_&ambiente..job_1363_pagamenti_cumulati;,[PRD_1363_COD_Lst_Stato_Pagamenti_Documenti_Fo...,1
7,set dri_&ambiente..lease_out_alpha_user ;,[PRD_1607_DRI_Lst_Billing_Check_FUELRCHRG_FUEL...,1
8,data fuel_issues; set dri_&ambiente..fuel_issu...,[PRD_1607_DRI_Lst_Billing_Check_FUELRCHRG_FUEL...,1


### FROM (difficile da ottenere...molte subquery e solitamente è presente solo FROM nella stringa)

In [14]:
df3_pgm, df3_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, r'\bfrom\b\s+\S+\s',[])

In [15]:
df3_linee

,\bfrom\b\s+\S+\s,programmi,Frequenza pgm
0,select * from connection to con1,[PRD_2070_Lista_Modelli_Veicoli_CarConfigurato...,13
31,select * from connection to ora1,[PRD_1394_DRI_Lst_Quotation_WLTP_Manual_v1.0.s...,9
34,from db a,[PRD_1312_DRI_Lst_Cofor_Check_Pagamenti_v1.0.s...,3
32,select * from connection to con6,[PRD_1312_DRI_Lst_Cofor_Check_Pagamenti_v1.0.s...,3
103,left join (select distinct lease_id from prest...,[PRD_1594_DRI_Lst_Inclusione_Veicoli_ArvalCars...,2
...,...,...,...
40,"(select *, count(*) as tot_studi from dbrichie...",[PRD_1315_DRI_Lst_Daily_Backlog_Studi_Risk_v2....,1
39,"(select *, count(*) as tot_studi from dbrichie...",[PRD_1315_DRI_Lst_Daily_Backlog_Studi_Risk_v2....,1
38,from dri_prd.crchreq_decisions aa,[PRD_1315_DRI_Lst_Daily_Backlog_Studi_Risk_v2....,1
37,from dri_prd.sub_codes sc,[PRD_1315_DRI_Lst_Daily_Backlog_Studi_Risk_v2....,1
